<a href="https://colab.research.google.com/github/VitorDalpino/automacao_python_unaerp/blob/main/desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importar bibliotecas
import pandas as pd
df_estoque_minimo = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Automacao_unaerp/Desafio/arquivos_base/EstoqueMin.csv', sep=';')
df_estoque_minimo['ID Produto'] = df_estoque_minimo['ID Produto'].astype(str)

df_estoque_minimo

# importar base de dados
df_base_dados = pd.ExcelFile('/content/drive/MyDrive/Colab Notebooks/Automacao_unaerp/Desafio/arquivos_base/BaseDados.xlsx')
df_base_dados.sheet_names

# Separar dados da base de dados
df_estoque = df_base_dados.parse('fEstoque')
df_loja = df_base_dados.parse('dLoja')
df_produto = df_base_dados.parse('dProduto')

df_estoque['ID Produto'] = df_estoque['ID Produto'].astype(str)
df_estoque['ID Loja'] = df_estoque['ID Loja'].astype(str)
df_produto['ID Produto'] = df_produto['ID Produto'].astype(str)

In [ ]:
# 1
estoque_atual = df_estoque.groupby('ID Produto')['Movimentação'].sum()
# Estoque_atual.columns = ['ID Produto','Estoque Atual']
# estoque_atual.rename(columns={'Movimentação': 'Estoque atual'}, inplace=True)#dicionario
estoque_atual = estoque_atual.rename('Estoque Atual')
estoque_atual.to_csv('/content/drive/MyDrive/Colab Notebooks/Automacao_unaerp/Desafio/respostas/estoque_atual.csv', sep=';')

In [ ]:
# Adicionar estoque mínimo no estoque atual
analise_estoque = pd.merge(estoque_atual, df_estoque_minimo, on= 'ID Produto', how = 'inner')

#Itens críticos
itens_criticos = analise_estoque[analise_estoque['Estoque Atual'] < analise_estoque['Estoque Mínimo']]
itens_criticos['Diferença Estoque']= itens_criticos['Estoque Atual'] - itens_criticos['Estoque Mínimo']

# Quantidade a comprar
# Corrected line: Using itens_criticos instead of intens_criticos
itens_criticos['Quantidade a Comprar'] = itens_criticos['Diferença Estoque'] * -1 + itens_criticos['Estoque Mínimo']

# Adicionar preço nos itens críticos
intens_criticos = pd.merge(itens_criticos, df_produto, on='ID Produto', how='inner')

# Corrected line: Selecting columns from intens_criticos instead of itens_criticos
intens_criticos[['ID Produto','Estoque Atual','Estoque Mínimo', 'Diferença Estoque', 'Quantidade a Comprar','Produto','Custo Unit']]

<ipython-input-6-531850dccb8e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  itens_criticos['Diferença Estoque']= itens_criticos['Estoque Atual'] - itens_criticos['Estoque Mínimo']
<ipython-input-6-531850dccb8e>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  itens_criticos['Quantidade a Comprar'] = itens_criticos['Diferença Estoque'] * -1 + itens_criticos['Estoque Mínimo']


,ID Produto,Estoque Atual,Estoque Mínimo,Diferença Estoque,Quantidade a Comprar,Produto,Custo Unit
0,10,167,168,-1,169,CAMISETA FITNESS MASCULINA FST500 VERDE CAQUI,23.99
1,12,109,118,-9,127,CAMISETA FITNESS MASCULINA FST500 AZUL/AZUL MA...,17.99
2,14,22,23,-1,24,CAMISETA FITNESS MASCULINA FST120 ESTAMPADO CAQUI,14.99
3,17,125,128,-3,131,CASACO MASCULINO GINÁSTICA SPACER,56.99
4,22,36,38,-2,40,TÊNIS FITNESS MASCULINO VERMELHO/PRETO LINHA 500,26.99
5,23,59,63,-4,67,TÊNIS FITNESS MASCULINO PRETO/CINZA LINHA 500,26.99
6,24,333,356,-23,379,TÊNIS FITNESS MASCULINO PRETO LINHA 100,35.99
7,25,17,18,-1,19,TÊNIS FITNESS MASCULINO PRETO LINHA 900,74.99
8,30,136,149,-13,162,LUVAS ACADEMIA / MEIA PALMA MUSCULAÇÃO NEOPREN...,7.79
9,33,0,20,-20,40,BOLSA FITNESS POWER 55 LITROS PRETA DOMYOS,44.99
